In [4]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='76dc74a2-d33c-4acb-8d27-dc3ddf423102', project_access_token='p-185703585a3d53d4fc6f817e963af8c4f6a1d361')
pc = project.project_context
print(project)
print(pc)

# Watson Visual Recognition Model usage... 
## work in progress

In [8]:
# check Python version
!python -V
# pip install --upgrade "watson-developer-cloud>=2.4.1"
# pip install --upgrade "ibm-watson>=3.0.4"
!pip -q install watson-developer-cloud==2.5.1
!pip install paho-mqtt

Python 3.6.8 :: Anaconda, Inc.
     |████████████████████████████████| 92kB 343kB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/82/e5/de/d90d0f397648a1b58ffeea1b5742ac8c77f71fd43b550fa5a5
Successfully built paho-mqtt


In [9]:
# Run the command if ibm_boto3 is not installed.
# !pip install ibm-cos-sdk

In [10]:
# Install the boto library.
import ibm_boto3
from ibm_botocore.client import Config

In [11]:
# Define endpoint information.
service_endpoint = 'https://s3-api.us-geo.objectstorage.softlayer.net'

In [12]:
# Define the authorization endpoint.
auth_endpoint = 'https://iam.bluemix.net/oidc/token'

In [13]:
# @hidden_cell
cos_credentials = {
  "apikey": "2ipjO2frCyVovFvWczZbm2KhLYqNXv5hSahxT193Oi7g",
  "cos_hmac_keys": {
    "access_key_id": "d8bce4166a0f4b17b05b5ac45a9e94f4",
    "secret_access_key": "758b7bbeda4987a847af9caa406eb5d0ef0476e5bca66662"
  },
  "endpoints": "https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints",
  "iam_apikey_description": "Auto-generated for key d8bce416-6a0f-4b17-b05b-5ac45a9e94f4",
  "iam_apikey_name": "HawkEyeServiceCredentials1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/64704b3c4d01439bad96805ae9cac499::serviceid:ServiceId-b0dcd823-b177-4fb8-b203-db6210e9b925",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/64704b3c4d01439bad96805ae9cac499:62dae09c-b741-461f-bb93-7f1fa7aa5aed::"
}

In [14]:
# Create a COS resource.
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=cos_credentials['apikey'],
                         ibm_service_instance_id=cos_credentials['resource_instance_id'],
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

In [15]:
# Display a list of created buckets.
print(list(cos.buckets.all()))

[s3.Bucket(name='hawkeyeflooddetection-donotdelete-pr-9dirx1ijye8ozp'), s3.Bucket(name='sql-7f499951-1ba1-41e5-bb4e-acdfdf6d1afb'), s3.Bucket(name='training-mnist-data-1bc6cde3-e15c-448f-accb-2516b508f43b'), s3.Bucket(name='training-mnist-results-1bc6cde3-e15c-448f-accb-2516b508f43b')]


In [ ]:
import json
import base64
import calendar
import time

import watson_developer_cloud
from watson_developer_cloud import VisualRecognitionV3

import paho.mqtt.client as mqtt

In [ ]:
"""
JSON Dumps Encoder
"""
class MyEncoder(json.JSONEncoder):
        def default(self, o):
            return o.__dict__

In [ ]:
"""
IBM Watson Visual Recognition Object
"""
visual_recognition = VisualRecognitionV3(
    version    = '2018-03-19',
    iam_apikey = 'K8-xMjYk_BdKRR5DqrVKovBmso4Icxi7QEv3nVTEGo-f',
)

In [ ]:
"""
MQTT Subscription
"""
# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    #print("Connected with result code "+str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe("incoming/pictures/#")
    with open("/watson/hackaton/images/flood.jpg", "rb") as flood_image:
	    encoded_flood_image = base64.b64encode(flood_image.read())
    with open("/watson/hackaton/images/sand.jpg", "rb") as sand_image:
	    encoded_sand_image = base64.b64encode(sand_image.read())
    client.publish("incoming/pictures/india", payload=encoded_flood_image)
    client.publish("incoming/pictures/india", payload=encoded_sand_image)
    print("Images published!")


In [ ]:
"""
IOT messages treatment
"""
# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    encoded_image = msg.payload
    # Stores the image
    with open("/watson/hackaton/images/image.jpg", "wb") as iot_image:
        iot_image.write(base64.decodebytes(encoded_image))
    print("Image received!")
	###########################################################################
	# Here we have to code the extraction of the image metadata (geolocation) #
	###########################################################################
    # Sends the image to Watson
    with open('/watson/hackaton/images/image.jpg', 'rb') as iot_image:
        response = visual_recognition.classify(iot_image, threshold=0.0, owners=["me"])
        response_dump = json.dumps(response, cls=MyEncoder, indent=2)
        parser = json.loads(response_dump)
        print("Image sent to Watson!")
        # Flood Class
        image_class = parser["result"]["images"][0]["classifiers"][0]["classes"][0]["class"]
        # Flood Level
        image_score = str(parser["result"]["images"][0]["classifiers"][0]["classes"][0]["score"])
        timestamp = calendar.timegm(time.gmtime())
        print(image_score)
        if float(image_score) > 0.6:
            print("Image has score greater than 0.6. Alert sent to the chat!")
            client.publish("users/chatbot_hawkeye", payload="{\"timestamp\":\"" + str(timestamp) + "\"}", qos=0, retain=False)
            client.publish("chat", payload="{\"clientId\": \"chatbot_hawkeye\", \"textReply\": \"Flood Alert!\nLatitude : 17.419310, Longitude : 78.337980\nLink : \"}", qos=0, retain=False)

In [ ]:
client = mqtt.Client("HawkEyes")
client.on_connect = on_connect
client.on_message = on_message

client.connect("frparvm-ion01", 1883, 60)

In [ ]:
# @hidden_cell
from ibm_watson import VisualRecognitionV3

visual_recognition = VisualRecognitionV3(
    version    = '2018-03-19',
    iam_apikey = 'K8-xMjYk_BdKRR5DqrVKovBmso4Icxi7QEv3nVTEGo-f'
)

# SQL Installation

In [5]:
!pip install jupyter-sql
!pip install ipython-sql

  Stored in directory: /home/dsxuser/.cache/pip/wheels/80/34/1c/3967380d9676d162cb59513bd9dc862d0584e045a162095606
Successfully built prettytable


# SQL

In [6]:
!pip -q install ibmcloudsql
!pip -q install sqlparse

In [7]:
import ibmcloudsql
from pixiedust.display import *
import pandas as pd
targeturl=''

Pixiedust database opened successfully
Table VERSION_TRACKER created successfully
Table METRICS_TRACKER created successfully

Share anonymous install statistics? (opt-out instructions)

PixieDust will record metadata on its environment the next time the package is installed or updated. The data is anonymized and aggregated to help plan for future releases, and records only the following values:

{
   "data_sent": currentDate,
   "runtime": "python",
   "application_version": currentPixiedustVersion,
   "space_id": nonIdentifyingUniqueId,
   "config": {
       "repository_id": "https://github.com/ibm-watson-data-lab/pixiedust",
       "target_runtimes": ["Data Science Experience"],
       "event_id": "web",
       "event_organizer": "dev-journeys"
   }
}
You can opt out by calling pixiedust.optOut() in a new cell.


Pixiedust runtime updated. Please restart kernel
Table USER_PREFERENCES created successfully
Table service_connections created successfully


In [8]:
cos_bucket = project.get_metadata()['entity']['storage']['properties']
print(cos_bucket)
targeturl="cos://" + cos_bucket['bucket_region'] + "/" + cos_bucket['bucket_name'] + "/"
print(targeturl)
targeturl="cos://us-south/sql-7f499951-1ba1-41e5-bb4e-acdfdf6d1afb/"
print(targeturl)

{'bucket_name': 'hawkeyeflooddetection-donotdelete-pr-9dirx1ijye8ozp', 'bucket_region': 'eu-geo', 'endpoint_url': 'https://s3.eu-geo.objectstorage.softlayer.net', 'credentials': {'editor': {'api_key': 'I0xWNZpkJjnwnsdTDoNnQbIC-Q9OoVpPu_nOjnDJ3QXm', 'service_id': 'iam-ServiceId-f52507e7-d5ec-4e9a-96fb-9bea2bc7c2ee', 'access_key_id': 'b58a12131744416a85c093621cb5d89a', 'secret_access_key': '1127f4e312180555a246ef35cadab967ef432c1d9a8355e6', 'resource_key_crn': 'crn:v1:bluemix:public:cloud-object-storage:global:a/64704b3c4d01439bad96805ae9cac499:62dae09c-b741-461f-bb93-7f1fa7aa5aed:resource-key:b58a1213-1744-416a-85c0-93621cb5d89a'}, 'viewer': {'api_key': 'VyCt__Ci-cTg8tiZ89WNLES6Zfkaw-C76JrvrnssXJRt', 'service_id': 'iam-ServiceId-6dda2db4-f3b2-4728-9039-9bf9dbf03d1a', 'access_key_id': '35b54aecbd17452a98074443228a1683', 'secret_access_key': 'ba81c9439e1d431f1da7c11a31c18eb5ccf42ac158514ab0', 'resource_key_crn': 'crn:v1:bluemix:public:cloud-object-storage:global:a/64704b3c4d01439bad96805a

In [9]:
apikey = "_gjUNa-8nefhYqCSR3HGdkHTUcTiOFN6Slz-iAioBhA0"
instnacecrn = "crn:v1:bluemix:public:sql-query:us-south:a/64704b3c4d01439bad96805ae9cac499:7f499951-1ba1-41e5-bb4e-acdfdf6d1afb::"
# API Endpoint	https://api.sql-query.cloud.ibm.com

In [10]:
import getpass
# apikey=getpass.getpass('Enter IBM Cloud API Key (leave empty to use previous one): ') or apikey
# instnacecrn=input('Enter SQL Query Instance CRN (leave empty to use previous one): ') or instnacecrn
if targeturl == '':
    targeturl=input('Enter target URL for SQL results: ')
# else:
  #  targeturl=input('Enter target URL for SQL results (leave empty to use ' + targeturl + '): ') or targeturl
sqlClient = ibmcloudsql.SQLQuery(apikey, instnacecrn, targeturl, client_info='SQL Query Starter Notebook')
# https://us.sql-query.cloud.ibm.com/sqlquery/?instance_crn=crn%3Av1%3Abluemix%3Apublic%3Asql-query%3Aus-south%3Aa%2F64704b3c4d01439bad96805ae9cac499%3A7f499951-1ba1-41e5-bb4e-acdfdf6d1afb%3A%3A
sqlClient.logon()
print('\nYour SQL Query web console link:\n')
sqlClient.sql_ui_link()


Your SQL Query web console link:

https://sql.ng.bluemix.net/sqlquery/?instance_crn=crn:v1:bluemix:public:sql-query:us-south:a/64704b3c4d01439bad96805ae9cac499:7f499951-1ba1-41e5-bb4e-acdfdf6d1afb::


In [11]:
import sqlparse
from pygments import highlight
from pygments.lexers import get_lexer_by_name
from pygments.formatters import HtmlFormatter, Terminal256Formatter

# sql=input('Enter your SQL statement (leave empty to use a simple sample SQL)')
# sql = 'SELECT * FROM cos://us-south/sql-7f499951-1ba1-41e5-bb4e-acdfdf6d1afb/Flood_Level.csv STORED AS CSV LIMIT 5'
# sql = 'SELECT * FROM cos://us-south/sql-7f499951-1ba1-41e5-bb4e-acdfdf6d1afb/Subscriber.csv STORED AS CSV LIMIT 50'
# sql = "SELECT EMAILID FROM cos://us-south/sql-7f499951-1ba1-41e5-bb4e-acdfdf6d1afb/Subscriber.csv  STORED AS CSV WHERE LEVEL < '3' LIMIT 50"
sql = "SELECT * FROM cos://us-south/sql-7f499951-1ba1-41e5-bb4e-acdfdf6d1afb/Subscriber.csv  STORED AS CSV WHERE SubscriberId = 1 LIMIT 50"

if sql == '':
    sql='SELECT o.OrderID, c.CompanyName, e.FirstName, e.LastName FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, cos://us-geo/sql/employees.parquet STORED AS PARQUET e, cos://us-geo/sql/customers.parquet STORED AS PARQUET c WHERE e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > "1998-01-01" ORDER BY c.CompanyName'
formatted_sql = sqlparse.format(sql, reindent=True, indent_tabs=True, keyword_case='upper')
lexer = get_lexer_by_name("sql", stripall=True)
formatter = Terminal256Formatter(style='tango')
result = highlight(formatted_sql, lexer, formatter)
from IPython.core.display import display, HTML
print('\nYour SQL statement is:\n')
print(result)


Your SQL statement is:

SELECT *
FROM cos://us-south/sql-7f499951-1ba1-41e5-bb4e-acdfdf6d1afb/Subscriber.csv  STORED AS CSV WHERE SubscriberId = 1 LIMIT 50



In [12]:
result_df = sqlClient.run_sql(sql)
if isinstance(result_df, str):
    print(result_df)
    print(result_df.EmailId[0])

In [13]:
result_df.head(10)

,SubscriberId,SubscriberType,Name,EmailId,Mobile,Whatsapp,Voice,Enroll,EnrollType,Level,Latitude,Longitude,Range,Review,Comments
0,1,'Authority','RanjanPatro',ranjan.patro@capgemini.com,917008897423,919908086992,0,True,12,1,17.41931,78.33798,10,'',''


# Email

In [14]:
#!/usr/bin/env python
# encoding: utf-8

import smtplib
from datetime import datetime

In [15]:
def noticeEMail( usr, psw, fromaddr, toaddr):
        # Initialize SMTP server
    server=smtplib.SMTP('smtp.gmail.com:587')
    server.starttls()
    server.login(usr,psw)
      # Send email
    senddate=datetime.strftime(datetime.now(), '%Y-%m-%d')
    subject="Your job has completed"
    m="Date: %s\r\nFrom: %s\r\nTo: %s\r\nSubject: %s\r\nX-Mailer: My-Mail\r\n\r\n" % (senddate, fromaddr, toaddr, subject)
    msg='''Bweare of the flood situation'''
    server.sendmail(fromaddr, toaddr, m+msg)
    server.quit()

In [16]:
print(result_df.EmailId[0])
print(result_df.Mobile[0])
print(result_df.Whatsapp[0])
print(result_df.Enroll[0])
print(result_df.EnrollType[0])

ranjan.patro@capgemini.com
917008897423
919908086992
True
12


In [17]:
# @hidden_cell
    usr='hawkeye190729@gmail.com'
    psw='29719!kwah'

In [18]:
    # Fill these in with the appropriate info...
    fromaddr=usr
    # toaddr='patror.pstech@gmail.com'
    toaddr=result_df.EmailId[0]
    # toaddr=toaddr.repalce("\'","")
    print(toaddr)

    # Send notification email
    noticeEMail(usr, psw, fromaddr, toaddr)

ranjan.patro@capgemini.com


# SMS/Whatsapp

In [19]:
!pip install twilio

     |████████████████████████████████| 1.0MB 303kB/s eta 0:00:01


In [20]:
# from twilio.rest import TwilioRestClient
from twilio.rest import Client

## SMS 

In [21]:
# @hidden_cell
    sid = "ACd2540e9538b40f444756d2ca1ab5b047" # "text-random"
    auth_token = "2173124db980319808416226a367e99b" # "youAreAuthed"
    twilio_number = "+17196242954" # "123-456-7890"   

In [22]:
 def send_sms(msg, to):
    """"""
 
 
    # client = TwilioRestClient(sid, auth_token)
    client = Client(sid, auth_token)
 
    message = client.messages.create(body=msg,
                                     from_=twilio_number,
                                     to=to,
                                     )
    print(message.sid)
    
    

## Whatsapp

In [23]:
# @hidden_cell
    account_sid = 'ACe7076dbddb8251103d96540258c499a7' 
    auth_token = 'e4bd2fa885e685b7ab1b4481a0a57cf5' 
    fromT='whatsapp:+14155238886'

In [24]:
# to_whatsapp_number=result_df.Whatsapp[0]
def send_whatsapp(msg, to_whatsapp_number):
    """"""

    client = Client(account_sid, auth_token) 
    print(to_whatsapp_number)
    message = client.messages.create( 
                          from_=fromT,                            
                          body=msg,      
                          to='whatsapp:+' + str(to_whatsapp_number)
                      ) 

    print(message.sid)

## Send SMS/Whatsapp 

In [28]:
if __name__ == "__main__":
    msg = "Please be vary of the flood situation near you!!"
    to_s = "+" + str(result_df.Mobile[0])
    to_w = result_df.Whatsapp[0]
    # send_sms(msg, to_s)
    send_whatsapp(msg,to_w)

919908086992
SM3025eb1be610467a84d09572f7d97c2c
